In [4]:
#11/24 과제
# 1. 강서구의 10개 : 가격 비교, 지도표시, 저렴한 곳은 빨강
# 2. 즐겨찾는(주변의) 주유소 10곳 : 가격 비교, 지도표시, 저렴한 곳은 빨강

In [8]:
import numpy as np
import pandas as pd 
from glob import glob
import requests
from urllib.parse import quote
from tqdm import tqdm_notebook
import json
import folium
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
#서울시의 주유소를 재인덱싱해서 엑셀로 만들고 불러옴.(crawling/10_opinet.ipynb 연결)
stations1 = pd.read_excel("../crawling/stations1.xlsx")
stations1

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
0,0,구천면주유소,서울 강동구 구천면로 357 (암사동),1374,N,현대오일뱅크,강동구,37.550458,127.138227
1,1,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1385,N,GS칼텍스,강동구,37.524535,127.134421
2,2,현대오일뱅크㈜직영 고속셀프주유소,서울 강동구 천호대로 1246,1386,Y,현대오일뱅크,강동구,37.536417,127.149372
3,3,방아다리주유소,서울 강동구 동남로 811 (명일동),1395,Y,SK에너지,강동구,37.546061,127.155495
4,4,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1395,Y,현대오일뱅크,강동구,37.554479,127.144011
...,...,...,...,...,...,...,...,...,...
482,482,갤러리아주유소,서울 강남구 압구정로 426,1848,N,SK에너지,강남구,37.526817,127.043532
483,483,학여울주유소,서울 강남구 남부순환로 3170 (일원2동),1858,N,GS칼텍스,강남구,37.498448,127.077572
484,484,(주)소모에너지엔테크놀러지 쎈트럴주유소,서울 강남구 삼성로 335,1883,N,GS칼텍스,강남구,37.501426,127.059074
485,485,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),1913,N,GS칼텍스,강남구,37.511551,127.047115


# 1번 :강서구의 10개

In [23]:
#강서구 중에서 골라보자.
stations2 =stations1[stations1['구'] == '강서구']
stations2 

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
316,316,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,N,알뜰주유소,강서구,37.531562,126.856035
317,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472
318,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436
319,319,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),1264,Y,현대오일뱅크,강서구,37.574341,126.802289
320,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029
321,321,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1265,Y,SK에너지,강서구,37.553947,126.768359
322,322,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,Y,GS칼텍스,강서구,37.529645,126.859835
323,323,목화주유소,서울 강서구 국회대로 251 (화곡동),1266,Y,SK에너지,강서구,37.530055,126.862834
324,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526
325,325,현대오일뱅크㈜직영 강서제일셀프주유소,서울 강서구 화곡로 273 (화곡동),1289,Y,현대오일뱅크,강서구,37.549279,126.846965


In [61]:
#강서구 저렴한 주유소 순
top10 = stations2.sort_values(by='가격', ascending=True).head(10)
top10

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
316,316,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,N,알뜰주유소,강서구,37.531562,126.856035
317,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472
318,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436
319,319,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),1264,Y,현대오일뱅크,강서구,37.574341,126.802289
320,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029
321,321,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1265,Y,SK에너지,강서구,37.553947,126.768359
322,322,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,Y,GS칼텍스,강서구,37.529645,126.859835
323,323,목화주유소,서울 강서구 국회대로 251 (화곡동),1266,Y,SK에너지,강서구,37.530055,126.862834
324,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526
325,325,현대오일뱅크㈜직영 강서제일셀프주유소,서울 강서구 화곡로 273 (화곡동),1289,Y,현대오일뱅크,강서구,37.549279,126.846965


In [62]:
#인덱스 재정렬
top10.reset_index(drop=True, inplace=True)
top10

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
0,316,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,N,알뜰주유소,강서구,37.531562,126.856035
1,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472
2,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436
3,319,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),1264,Y,현대오일뱅크,강서구,37.574341,126.802289
4,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029
5,321,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1265,Y,SK에너지,강서구,37.553947,126.768359
6,322,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,Y,GS칼텍스,강서구,37.529645,126.859835
7,323,목화주유소,서울 강서구 국회대로 251 (화곡동),1266,Y,SK에너지,강서구,37.530055,126.862834
8,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526
9,325,현대오일뱅크㈜직영 강서제일셀프주유소,서울 강서구 화곡로 273 (화곡동),1289,Y,현대오일뱅크,강서구,37.549279,126.846965


In [79]:
#전체 색깔지정
top10['color'] = 'blue'
top10

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng,color
0,316,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,N,알뜰주유소,강서구,37.531562,126.856035,blue
1,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472,blue
2,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436,blue
3,319,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),1264,Y,현대오일뱅크,강서구,37.574341,126.802289,blue
4,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029,blue
5,321,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1265,Y,SK에너지,강서구,37.553947,126.768359,blue
6,322,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,Y,GS칼텍스,강서구,37.529645,126.859835,blue
7,323,목화주유소,서울 강서구 국회대로 251 (화곡동),1266,Y,SK에너지,강서구,37.530055,126.862834,blue
8,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526,blue
9,325,현대오일뱅크㈜직영 강서제일셀프주유소,서울 강서구 화곡로 273 (화곡동),1289,Y,현대오일뱅크,강서구,37.549279,126.846965,blue


In [74]:
#젤 저렴한 곳 구하기
top1 = top10.loc[0] 
top1 = top10.sort_values(by='가격', ascending=True).head(1)
top1['color'] ='red'
top1

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng,color
0,316,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,N,알뜰주유소,강서구,37.531562,126.856035,red


In [106]:
top10.lat[0]

37.5315624983884

In [85]:
#10곳 지도 표시
mapping = folium.Map(
    location=[top10['lat'].mean(),top10['lng'].mean()], zoom_start=14)
for i in top10.index:
    folium.Marker(
    location=[top10['lat'][i], top10['lng'][i]],
    popup = f"주소 : {top10['주소'][i]},\n상표: {top10['상표'][i]}",
    
    tooltip =top10['상호'][i],
    icon=folium.Icon(color=top10.color[i], icon="info-sign")
).add_to(mapping)
mapping

In [100]:
#저렴한곳 지도 표시
mapping = folium.Map(
    location=[top1['lat'].mean(),top1['lng'].mean()], zoom_start=14)

for i in top1.index:
    folium.Marker(
    location=[top1['lat'][i], top1['lng'][i]],
    popup = f"주소 : {top1['주소'][i]},\n상표: {top1['상표'][i]}",
    
    tooltip =top1['상호'][i],
    icon=folium.Icon(color=top1.color[i], icon="info-sign")
).add_to(mapping)
mapping

In [109]:
top1['주소'][0]

'서울 강서구 곰달래로 207(화곡동)'

In [111]:
top1['lat'][0]

37.5315624983884

In [122]:
top1['가격'][0]

1243

In [113]:
#저렴한 곳 지도 표시 - 다른 방법
mapping = folium.Map(
    location=[top1['lat'][0].mean(),top1['lng'][0].mean()], zoom_start=14)


folium.Marker(
    location=[top1['lat'][0], top1['lng'][0]],
    popup = f"주소 : {top1['주소'][0]},\n상표: {top1['상표'][0]}",
    
    tooltip =top1['상호'][0],
    icon=folium.Icon(color=top1.color[0], icon="info-sign")
).add_to(mapping)
mapping

1. 각각 개별로 한번에 표시하는 방법

In [115]:
#한번에 표시하기
mapping = folium.Map(
    location=[top10['lat'].mean(),top10['lng'].mean()], zoom_start=14)
for i in top10.index:
    folium.Marker(
    location=[top10['lat'][i], top10['lng'][i]],
    popup = f"주소 : {top10['주소'][i]},\n상표: {top10['상표'][i]}",
    
    tooltip =top10['상호'][i],
    icon=folium.Icon(color=top10.color[i], icon="info-sign")
).add_to(mapping)

folium.Marker(
    location=[top1['lat'][0], top1['lng'][0]],
    popup = f"주소 : {top1['주소'][0]},\n상표: {top1['상표'][0]}",
    
    tooltip =top1['상호'][0],
    icon=folium.Icon(color=top1.color[0], icon="info-sign")
).add_to(mapping)
mapping

2.두개의 데이터 프레임을 병합하여 하는 방법

In [103]:
#일단 1번째열이 중복 되므로, 제거해준다.
station=top10.drop([0])
station

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng,color
1,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472,blue
2,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436,blue
3,319,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),1264,Y,현대오일뱅크,강서구,37.574341,126.802289,blue
4,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029,blue
5,321,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1265,Y,SK에너지,강서구,37.553947,126.768359,blue
6,322,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,Y,GS칼텍스,강서구,37.529645,126.859835,blue
7,323,목화주유소,서울 강서구 국회대로 251 (화곡동),1266,Y,SK에너지,강서구,37.530055,126.862834,blue
8,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526,blue
9,325,현대오일뱅크㈜직영 강서제일셀프주유소,서울 강서구 화곡로 273 (화곡동),1289,Y,현대오일뱅크,강서구,37.549279,126.846965,blue


In [104]:
#데이터 프레임 병합
stations = pd.concat([top1, station])
stations

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng,color
0,316,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,N,알뜰주유소,강서구,37.531562,126.856035,red
1,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472,blue
2,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436,blue
3,319,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),1264,Y,현대오일뱅크,강서구,37.574341,126.802289,blue
4,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029,blue
5,321,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1265,Y,SK에너지,강서구,37.553947,126.768359,blue
6,322,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,Y,GS칼텍스,강서구,37.529645,126.859835,blue
7,323,목화주유소,서울 강서구 국회대로 251 (화곡동),1266,Y,SK에너지,강서구,37.530055,126.862834,blue
8,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526,blue
9,325,현대오일뱅크㈜직영 강서제일셀프주유소,서울 강서구 화곡로 273 (화곡동),1289,Y,현대오일뱅크,강서구,37.549279,126.846965,blue


In [105]:
#병합된 데이터프레임으로 다시 지도에 한번에 표시하는 것
mapping = folium.Map(
    location=[stations['lat'].mean(),stations['lng'].mean()], zoom_start=14)
for i in stations.index:
    folium.Marker(
    location=[stations['lat'][i], stations['lng'][i]],
    popup = f"주소 : {stations['주소'][i]},\n상표: {stations['상표'][i]}",
    
    tooltip =stations['상호'][i],
    icon=folium.Icon(color=stations.color[i], icon="info-sign")
).add_to(mapping)
mapping

3. 최소값을 이용하는 방법

In [124]:
min(top10['가격'])

1243

In [145]:
mapping = folium.Map(location=[top10['lat'].mean(), top10['lng'].mean()], zoom_start=14)

for i in top10.index:
    location = [top10['lat'][i], top10['lng'][i]]
    popup = f"{top10['상호'][i]}"
    tooltip=f"{top10['가격'][i]}원, 주소:{top10['주소'][i]}"
    if top10['가격'][i] == min(top10['가격']):
        folium.Marker(
            location = location,
            tooltip=tooltip,  #마커안에 안들어가있으면, 지도에 표시가 안된다.
            popup=popup,
            icon=folium.Icon(color='red', icon='heart')
            ).add_to(mapping)
    else:
        folium.Marker(
            location =location,
            tooltip=tooltip,
            popup=popup,
            icon=folium.Icon(color='blue', icon='flag')
            ).add_to(mapping)
mapping

In [152]:
# 더 안줄여지네...ㅠㅠ  ??  물어보자!
mapping = folium.Map(location=[top10['lat'].mean(), top10['lng'].mean()], zoom_start=14)
for i in top10.index:
    location = [top10['lat'][i], top10['lng'][i]]
    popup = f"{top10['상호'][i]}"
    tooltip=f"{top10['가격'][i]}원, 주소:{top10['주소'][i]}"
    folium.Marker(
        location = location,
        tooltip=tooltip,
        popup=popup,
        if top10['가격'][i] == min(top10['가격']):
            icon=folium.Icon(color='red', icon='heart')
        else:
            icon=folium.Icon(color='blue', icon='flag')
    ).add_to(mapping)
mapping

SyntaxError: invalid syntax (<ipython-input-152-b7aa3e2fe54d>, line 11)

# 2. 즐겨찾는(주변의) 12곳

In [236]:
stations2

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
0,316,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,N,알뜰주유소,강서구,37.531562,126.856035
1,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472
2,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436
3,319,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),1264,Y,현대오일뱅크,강서구,37.574341,126.802289
4,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029
5,321,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1265,Y,SK에너지,강서구,37.553947,126.768359
6,322,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,Y,GS칼텍스,강서구,37.529645,126.859835
7,323,목화주유소,서울 강서구 국회대로 251 (화곡동),1266,Y,SK에너지,강서구,37.530055,126.862834
8,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526
9,325,현대오일뱅크㈜직영 강서제일셀프주유소,서울 강서구 화곡로 273 (화곡동),1289,Y,현대오일뱅크,강서구,37.549279,126.846965


In [234]:
#원하는 주유소 목록.
station_list = ['방화주유소','스카이주유소','개화주유소','현대오일뱅크㈜직영 에어포트주유소','방화대성주유소','마곡대성주유소','김포공항제2주유소','화곡역주유소','염창동주유소','유턴주유소','직영관문주유소','우장산주유소'] 

In [193]:
a = stations2.loc[stations2['상호'].str.contains('방화주유소', na=False)]
a

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
2,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436


In [216]:
station_list[-2], len(station_list),len(stations2)

('대성석유(주)직영관문주유소', 11, 33)

In [235]:
# 내 주유소만 골라서 데이타 프레임으로 생성
a = stations2.loc[stations2['상호'].str.contains(station_list[0], na=False)]

for i in range(1,len(station_list)):
    b = stations2.loc[stations2['상호'].str.contains(station_list[i], na=False)]
    a = pd.concat([a, b])  
a       


,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
2,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436
8,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526
11,327,개화주유소,서울 강서구 양천로 57,1297,Y,SK에너지,강서구,37.573547,126.810054
19,335,현대오일뱅크㈜직영 에어포트주유소,서울 강서구 개화동로 519 (방화동),1339,N,현대오일뱅크,강서구,37.566293,126.806040
22,338,대성석유(주)방화대성주유소,서울 강서구 양천로 176,1365,Y,GS칼텍스,강서구,37.572898,126.823440
23,339,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,1365,Y,GS칼텍스,강서구,37.566243,126.840421
30,346,(주)한진 김포공항제2주유소,서울 강서구 하늘길 112,1407,Y,S-OIL,강서구,37.558784,126.803355
1,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472
32,348,염창동주유소,서울 강서구 양천로 720 (염창동),1688,N,SK에너지,강서구,37.549218,126.875677
4,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029


In [238]:
my_station = a.sort_values(by='가격').reset_index(drop=True)
my_station

,Unnamed: 0,상호,주소,가격,셀프,상표,구,lat,lng
0,317,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,Y,알뜰주유소,강서구,37.540768,126.841472
1,318,방화주유소,서울 강서구 방화동로 58-2 (방화동),1263,N,SK에너지,강서구,37.565563,126.812436
2,320,유턴주유소,서울 강서구 남부순환로 57,1265,Y,알뜰주유소,강서구,37.557248,126.810029
3,324,스카이주유소,서울 강서구 개화동로 457 (방화동),1269,Y,S-OIL,강서구,37.571522,126.804526
4,327,개화주유소,서울 강서구 양천로 57,1297,Y,SK에너지,강서구,37.573547,126.810054
5,335,현대오일뱅크㈜직영 에어포트주유소,서울 강서구 개화동로 519 (방화동),1339,N,현대오일뱅크,강서구,37.566293,126.806040
6,336,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,1358,N,GS칼텍스,강서구,37.557842,126.842395
7,337,우장산주유소,서울 강서구 강서로 292 (내발산동),1358,N,SK에너지,강서구,37.551803,126.836681
8,338,대성석유(주)방화대성주유소,서울 강서구 양천로 176,1365,Y,GS칼텍스,강서구,37.572898,126.823440
9,339,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,1365,Y,GS칼텍스,강서구,37.566243,126.840421


In [245]:
 my_station['상호'][len(my_station)-1]

'염창동주유소'

In [249]:
n = len(my_station)-1
mapping = folium.Map(
    location=[my_station['lat'].mean(),my_station['lng'].mean()], zoom_start=14)
for i in my_station.index:
    folium.Marker(
    location=[my_station['lat'][i], my_station['lng'][i]],
    popup = f"주소 : {my_station['주소'][i]},\n상표: {my_station['상표'][i]}",
    
    tooltip =my_station['상호'][i],
    icon=folium.Icon(color='blue', icon="info-sign")
).add_to(mapping)

#저렴한 곳
folium.Marker(
    location=[my_station['lat'][0], my_station['lng'][0]],
    popup = f"주소 : {my_station['주소'][0]},\n상표: {my_station['상표'][0]}",
    
    tooltip =my_station['상호'][0],
    icon=folium.Icon(color='red', icon="info-sign")
).add_to(mapping)
#가장 비싼곳
folium.Marker(
    location=[my_station['lat'][n], my_station['lng'][n]],
    popup = f"주소 : {my_station['주소'][n]},\n상표: {my_station['상표'][n]}",
    
    tooltip =my_station['상호'][n],
    icon=folium.Icon(color='black', icon="info-sign")
).add_to(mapping)
mapping

